# Bài toán

> **Thu thập dữ liệu báo Việt Nam (VnExpress - Demo)**

Mục tiêu:

- Hiểu được quy trình thu thập dữ liệu từ các trang báo Việt Nam
- Thu thập dữ liệu (bài báo) từ các trang báo Việt Nam để làm dữ liệu cho các bước xử lý sau

Đầu ra:

-  Tập file JSON chứa các bài bài báo có các trường dữ liệu:

    - `url`: link dẫn đến bài báo
    - `title`: tiêu đề bài báo
    - `description`: tóm tắt bài báo
    - `content`: nội dung bài báo
    - `metadata`: trường dữ liệu bổ sung

        - `cat`: thể loại bài báo
        - `subcat`: thể loại con của bài báo
        - `published_date`: thời gian xuất bản
        - `author`: người viết
    
- Ví dụ về một bài báo:

    ```
    {
        "url": "https://vnexpress.net/chinh-phu-ban-hanh-nghi-dinh-moi-ve-gia-dat-4763835.html",
        "title": "Chính phủ ban hành nghị định mới về giá đất",
        "description": "Chính phủ hôm nay ban hành Nghị định 71, trong đó quy ...",
        "content": "Nghị định này có hiệu lực khi Luật Đất đai 2024 được thi hành ...",
        "metadata": {
            "cat": "Bất động sản",
            "subcat": "Chính sách",
            "published_date": 1719575647,
            "author": "Anh Tú"
        }
    },
    ```

# Các bước tiến hành

## Chuẩn bị các thư viện cần thiết

In [45]:
# Suitable for Google Colab, for local please follow the external instructions and ignore this line
# and follows https://docs.google.com/document/d/14jK9d6KHJYX0b-gFAVqAghUxT7OLAM0nP2IovL7_Rjs/edit?usp=sharing
!apt install -qq chromium-chromedriver

'apt' is not recognized as an internal or external command,
operable program or batch file.


In [46]:
# Install selenium
%pip install -qq selenium

# Tạo thư mục để chứa data
!mkdir data

Note: you may need to restart the kernel to use updated packages.


A subdirectory or file data already exists.


In [1]:
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# other imports
import os
import json

In [2]:
# selenium setups
## https://www.tutorialspoint.com/selenium/selenium_webdriver_chrome_webdriver_options.htm

chrome_options = webdriver.ChromeOptions()

# chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")


In [3]:
MAGAZINE_NAME = "vnexpress"
HOME_PAGE = "https://vnexpress.net/bat-dong-san"


# Thu thập urls

In [4]:
driver = webdriver.Chrome(options=chrome_options)

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


In [ ]:
driver.get(HOME_PAGE)

N_URLS = 10

urls = []
id = 0
page = 1
start = -1
cnt_err = 0
while len(urls) < N_URLS:
    id += 1
    print(f"id={id}")
    try:
        element = driver.find_element(By.CSS_SELECTOR, f'a[data-medium="Item-{id}"]')
        href = element.get_attribute("href")
        if start == -1:
            start = id
            print(f'page={page}, start={start}')
        print(href)
        urls.append(href)
        cnt_err = 0

    except (StaleElementReferenceException, NoSuchElementException) as e:
        cnt_err += 1
        if start != -1 and cnt_err == 3: # nếu đang crawl mà có 3 bài lỗi liên tục -> nhảy qua trang tiếp theo để kiếm
            id -= cnt_err
            print(f'Bug at id={id + 1}, next, find in page={page + 1}')
            print('Finding start_id.......')
            driver.get(HOME_PAGE + "-p" + str(page + 1))
            page += 1
            start = -1
            cnt_err = 0
        driver.refresh()
        # Chúng ta tạm bỏ lỗi bây giờ
        continue
    except Exception:
        print('Exception')
        continue



In [6]:
driver.close()
urls

['https://vnexpress.net/nha-ong-mot-mat-thoang-voi-du-khong-gian-san-vuon-4860582.html',
 'https://vnexpress.net/co-nen-ban-dat-vung-ven-de-dau-tu-can-ho-trung-tam-4859514.html',
 'https://vnexpress.net/80-lo-dat-trung-dau-gia-tai-ha-dong-bi-bo-coc-4860359.html',
 'https://vnexpress.net/tp-hcm-se-ho-tro-50-chi-phi-ha-tang-cai-tao-chung-cu-cu-4860348.html',
 'https://vnexpress.net/co-nen-ban-dat-tinh-de-doi-can-chung-cu-rong-hon-o-ha-noi-4860298.html',
 'https://vnexpress.net/cong-ty-con-hoa-phat-se-lam-9-000-can-nha-xa-hoi-tai-hung-yen-4860280.html',
 'https://vnexpress.net/dong-tho-khu-cong-nghiep-vsip-ii-quang-ngai-4860230.html',
 'https://vnexpress.net/phat-dat-dieu-chinh-doanh-thu-giam-1-200-ty-dong-4860059.html',
 'https://vnexpress.net/cong-ty-dung-ai-ban-bat-dong-san-thu-ve-100-trieu-usd-4859585.html',
 'https://vnexpress.net/khoi-cong-khu-cong-nghiep-sinh-thai-400-ha-phia-tay-tp-hcm-4860514.html']

## Thu thập dữ liệu

> **Các bước thu thập bài báo**


1. News categories: Thu thập tất cả các thể loại báo của website
2. News urls: Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó
3. News articles: Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước

### Thu thập thể loại bài báo của website: Crawling categories

> **Các bước thu thập**

1. Vào trang chủ của báo
2. Thu thập các thể loại ở mục menu


Vào trang chủ

In [7]:
driver = webdriver.Chrome(options=chrome_options)
# Vào trang web chính, mặc định phải chờ toàn bộ trang webload mới xong
driver.get(HOME_PAGE)

Chọn menu buttons

Thu thập hết thể loại:

* Cách chọn elements ở web trong selenium: https://selenium-python.readthedocs.io/locating-elements.html

### Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước: News articles


> **Cách thu thập**

Từ đường dẫn ở trong phần trước, ta lần lượt vào từng đường link đó và thu thập thông tin về bài báo.

#### Cài đặt thông số

In [8]:
# Filepath cho cái trước
FILE_URL_PATH = "data/vnexpress_url.json"

# Data output, mỗi thể loại là 1 file json chứa articles
DATA_FOLDER_OUTPUT = "data\\vnexpress"
#
os.system("md " + DATA_FOLDER_OUTPUT)

# Để loading strategy về eager load nhanh, không quan tâm ảnh
chrome_options.page_load_strategy = "eager"

#### Chạy thử nghiệm

In [11]:
driver = webdriver.Chrome(options=chrome_options)

# Một số url để thử nghiệm
SAMPLE_ARTICLE_URLS = [
    "https://vnexpress.net/tp-hcm-cap-so-hong-cho-41-du-an-nha-o-thuong-mai-4839917.html",
    "https://vnexpress.net/thu-tuong-yeu-cau-nghien-cuu-danh-thue-dau-co-bat-dong-san-4839559.html"
]

# chọn url
SAMPLE_ARTICLE_URL = SAMPLE_ARTICLE_URLS[-1]
SAMPLE_ARTICLE_URL

'https://vnexpress.net/thu-tuong-yeu-cau-nghien-cuu-danh-thue-dau-co-bat-dong-san-4839559.html'

In [12]:
driver.get(SAMPLE_ARTICLE_URL)

In [13]:
# Tìm kiếm title
driver.find_element(by=By.CSS_SELECTOR, value="h1.title-detail").text

'Thủ tướng yêu cầu nghiên cứu đánh thuế đầu cơ bất động sản'

In [14]:
# Tìm kiếm description
driver.find_element(by=By.CLASS_NAME, value="description").text

'Thủ tướng Phạm Minh Chính yêu cầu Bộ Tài chính nghiên cứu, đề xuất chính sách thuế nhằm hạn chế hoạt động đầu cơ bất động sản.'

In [ ]:
# Thu thập thể loại
lis_cat = driver.find_element(by=By.CSS_SELECTOR, value="ul.breadcrumb").find_elements(by=By.TAG_NAME, value="li")
main_cat = lis_cat[0].text if len(lis_cat) > 0 else None
sub_cat = lis_cat[1].text if len(lis_cat) > 1 else None
main_cat, sub_cat

Bất động sản Chính sách 

('Bất động sản', 'Chính sách')

In [ ]:
# Thu thập ngày
publish_date = driver.find_element(by=By.CSS_SELECTOR, value='[itemprop="datePublished"]').get_attribute("content").strip()
publish_date

'2025-01-16T00:02:00+07:00'

In [ ]:
# Tìm kiếm contents
article = driver.find_element(by=By.CSS_SELECTOR, value="article.fck_detail")
children = article.find_elements(by=By.XPATH, value="./*")

In [ ]:
# Thu thập contents và author
contents = []
author = "Unknown"
is_slide_show = False

for idx, child in enumerate(children):
    text = child.text.strip()
    # right align
    if child.tag_name == "p" and ("right" in child.get_attribute("align") or "right" in child.get_attribute("style")) and idx >= len(children) - 3: # last three, align right --> author
        author = text
    elif child.tag_name == "p" and child.get_attribute("class") == "Normal": # paragraph
        # If center
        if len(text):
            if ("center" in child.get_attribute("align") or "center" in child.get_attribute("style")):
                contents.append(f"[{text}]")
            else:
                contents.append(text)
    elif child.tag_name == "figure" :
        ## If length > 100  --> not a caption, it's next description
        if len(text):
            if len(text) <= 100:
                contents.append(f"[{text}]")
            else:
                contents.append(text)
    elif child.tag_name == "div" and "item_slide_show" in child.get_attribute("class"):
        is_slide_show = True # slideshow
        if len(text):
            if len(text) <= 100:
                contents.append(f"[{text}]")
            else:
                contents.append(text)

    elif child.tag_name == "table": # Do nothing rightnow
        pass

if is_slide_show:
    author = text

if author == "Unknown":
    try:
        author = driver.find_element(by=By.XPATH, value="//*[contains(@class, 'author')]").text
    except:
        pass

In [ ]:
contents

['Nội dung trên được Thủ tướng Phạm Minh Chính đề cập trong công điện gửi các bộ ngành, địa phương, yêu cầu tập trung chấn chỉnh, xử lý việc thao túng giá, đầu cơ bất động sản và thanh tra, kiểm tra các dự án đầu tư xây dựng bất động sản.',
 'Theo đó, trong năm 2024, một số khu vực ghi nhận giá bất động sản tăng cao so với khả năng đáp ứng tài chính của người dân. Nguyên nhân là một số hội, nhóm đầu cơ lợi dụng sự thiếu hiểu biết, tâm lý đầu tư theo đám đông của người dân để thao túng tâm lý, "đẩy giá tăng cao", "tạo giá ảo" gây nhiễu loạn thông tin thị trường nhằm trục lợi.',
 'Ngoài ra, một số chủ đầu tư dự án bất động sản lợi dụng tình hình nguồn cung bất động sản hạn chế để đưa ra giá chào bán cao hơn mức trung bình của các dự án để thu lợi. Kết quả trúng đấu giá quyền sử dụng đất cao bất thường tại một số khu vực làm tăng mặt bằng giá đất, giá nhà ở.',
 'Để tăng cường kiểm soát và xử lý kịp thời việc thao túng, đẩy giá và đầu cơ bất động sản, Thủ tướng yêu cầu Bộ trưởng Tài chính 

In [ ]:
author

'Unknown'

In [15]:
driver.close()

#### Chạy thật


In [21]:
def get_content_metadata(driver, article_url):

    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :return: Dictionary containing article metadata and content.
    """

    # Get to current article
    driver.get(article_url)

    # Thu thập title
    title = driver.find_element(by=By.CSS_SELECTOR, value="h1.title-detail").text.strip()

    # Thu thập description
    description = driver.find_element(by=By.CLASS_NAME, value="description").text.strip()

    # Thu thập thể loại
    lis_cat = driver.find_element(by=By.CSS_SELECTOR, value="ul.breadcrumb").find_elements(by=By.TAG_NAME, value="li")
    main_cat = lis_cat[0].text if len(lis_cat) > 0 else None
    sub_cat = []
    for i in range(1, len(lis_cat)):
        sub_cat.append(lis_cat[i].text)
    if len(sub_cat) == 0:
        sub_cat =  None
    

    # Thu thập published date
    publish_date = driver.find_element(by=By.CSS_SELECTOR, value='[itemprop="datePublished"]').get_attribute("content").strip()

    # Thu thập content bài báo
    # Locate phần viết content
    article = driver.find_element(by=By.CSS_SELECTOR, value="article.fck_detail")
    # Lấy hết các đầu mục con của bài báo
    children = article.find_elements(by=By.XPATH, value="./*")

    contents = []
    author = "Unknown"

    # Check có phải dạng slide show hay không
    is_slide_show = False
    for idx, child in enumerate(children):
        text = child.text.strip()
        # Nếu mà element right align --> có thể là tác giả
        if child.tag_name == "p" and ("right" in child.get_attribute("align") or "right" in child.get_attribute("style")) and idx >= len(children) - 3: # last three, align right --> author
            author = text
        elif child.tag_name == "p" and child.get_attribute("class") == "Normal": # paragraph
            # If center
            if len(text):
                if ("center" in child.get_attribute("align") or "center" in child.get_attribute("style")):
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Chỉ lấy caption của figure
        elif child.tag_name == "figure" :
            ## If length > 100  --> not a caption, it's next description
            if len(text):
                if len(text) <= 100: # nếu mà len <= 100 --> add thêm [] xung quanh
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Nếu mà là slide show thì nó giống figure
        elif child.tag_name == "div" and "item_slide_show" in child.get_attribute("class"):
            is_slide_show = True # slideshow
            if len(text):
                if len(text) <= 100:
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Bỏ qua table bây giờ
        elif child.tag_name == "table": # Do nothing rightnow
            pass

    if is_slide_show:
        author = text

    # Nếu mà vẫn chưa thấy author thì tìm bằng tag
    if author == "Unknown":
        try:
            author = driver.find_element(by=By.XPATH, value="//*[contains(@class, 'author')]").text
        except:
            pass

    return {
        "url": article_url,
        "title": title,
        "description": description,
        "content": "\n".join(contents), # join các đoạn bằng \n
        "metadata": {
            "cat": main_cat,
            "subcat": sub_cat,
            "published_date": publish_date,
            "author": author
        }
    }


In [24]:
driver = webdriver.Chrome(options=chrome_options)

count_crawled = 0
cat_data = []

print(f"Thu thập dữ liệu...")
for id_url, url in enumerate(urls):
    print(f"id_url={id_url}")
    try:
        cat_data.append(get_content_metadata(driver, url))
        count_crawled += 1

    except (StaleElementReferenceException, NoSuchElementException) as e:
        print(f"Bug at url: {url}, with ElementException")
        driver.refresh()
        # Chúng ta tạm bỏ lỗi bây giờ
        continue
    except Exception:
        print('Exception')
        continue

name_file_cat = 'vnexpress_bds.json'

with open(os.path.join(DATA_FOLDER_OUTPUT, name_file_cat), "w", encoding='utf-8') as fOut:
    json.dump(cat_data, fOut, ensure_ascii=False, indent=4)

driver.close()

Thu thập dữ liệu...
id_url=0
id_url=1
id_url=2
id_url=3
id_url=4
id_url=5
id_url=6
id_url=7
id_url=8
id_url=9


In [23]:
# Xem 1 sample
cat_data[0]

{'url': 'https://vnexpress.net/tp-hcm-sap-dau-gia-khu-dat-26-ha-lam-tod-gan-metro-so-2-4858952.html',
 'title': 'TP HCM sắp đấu giá khu đất 26 ha làm TOD gần Metro số 2',
 'description': 'Cuối năm nay, TP HCM sẽ đấu giá khu đất 26 ha tại quận Tân Phú để làm TOD dọc tuyến Metro số 2 (Bến Thành - Tham Lương).',
 'content': 'TOD là mô hình lấy định hướng phát triển giao thông công cộng để quy hoạch, xây dựng và phát triển đô thị. Những nơi làm TOD được tăng hệ số sử dụng đất, mật độ dân số cao, từ đó thu hút nhu cầu đi và đến các đầu mối giao thông lớn. Mô hình trên đã triển khai ở nhiều nơi nhưng trong nước chưa có.\nMới đây, UBND quận Tân Phú ban hành kế hoạch triển khai khu vực TOD dọc tuyến Metro số 2 (Bến Thành - Tham Lương) trong năm 2025. Thông qua đánh giá hiện trạng và quy hoạch, quận Tân Phú đã chọn khu đất số I/82 A phường Tây Thạnh là nơi tạo quỹ đất để triển khai mô hình đô thị nén.\nKhu đất I/82 A Tây Thạnh có diện tích 26 ha, nằm tiếp giáp hai trục đường Tây Thạnh và Trường

## Lưu dữ liệu

Nếu bạn chạy ở máy cá nhân thì không cần, nhưng nếu mà chạy ở Colab thì nên lưu dữ liệu vào trong Google Drive


In [ ]:
# # For Google Colab
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# # Set to your folder
# FOLDER_SAVED_GOOGLE_COLAB = "/content/drive/MyDrive/crawl-news/"

# # Copy
# !cp -r data $FOLDER_SAVED_GOOGLE_COLAB